In [1]:
### Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import copy
import time
import os
import sys

#import nltk

In [2]:
### Custom Imports
sys.path.append('../')
import lib.utilities as utils

In [3]:
corpus = utils.load_corpus()

In [4]:
test = utils.preprocess_text(corpus, keep='\<|\>')

In [5]:
test1 = utils.split_text(corpus)

In [6]:
test2 = utils.split_song(test1[1])

In [7]:
test1[1]

"<VERSE>\nIn the early years Our poor pioneers\nWorked down by the riverside\nDrawn from many lands With dreams in their hands\nThe values they gave us survive\n\n<VERSE>\nAs four streams flow To make one river grow\nSo we came together as one\nWe live side by side And share island wide\nA style Singaporean\n\n<CHORUS>\nThis is our home A home that's grown\nFrom a little room To a country of our own\nNow we welcome everyone to see\nSingapore - a city for the world\n\n<VERSE>\nWe reach out our hands To friends in far off lands\nAnd bridge dividing streams\nTogether we can build A world which is filled\nWith hope of the Singapore dream\n\n<CHORUS>\nThis is our home A home that's grown\nFrom a little room To a country of our own\nNow we welcome everyone to see\nSingapore - a city for the world\n\n<CHORUS>\nThis is our home A home that's grown\nFrom a little room To a country of our own\nNow we welcome everyone to see\nSingapore - a city for the world\n\n<CHORUS>\nThis is our home A home t

In [8]:
test2

{'<BRIDGE>': [],
 '<CHORUS>': ["This is our home A home that's grown\nFrom a little room To a country of our own\nNow we welcome everyone to see\nSingapore - a city for the world",
  "This is our home A home that's grown\nFrom a little room To a country of our own\nNow we welcome everyone to see\nSingapore - a city for the world"],
 '<OTHERS>': [],
 '<PRECHORUS>': [],
 '<PRELUDE>': [],
 '<VERSE>': ['In the early years Our poor pioneers\nWorked down by the riverside\nDrawn from many lands With dreams in their hands\nThe values they gave us survive',
  'We reach out our hands To friends in far off lands\nAnd bridge dividing streams\nTogether we can build A world which is filled\nWith hope of the Singapore dream']}